In [5]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from api_key_file import api_key
from IPython.display import JSON
import requests
import json
import pandas as pd
import csv

api_service_name = "youtube"
api_version = "v3"

In [6]:
channel_ids = ["UCLKZ20yD2tNMBOkSDZo4FeQ",
"UCzAF54cHk1ZO82af-8E3qOQ",
"UCteRPiisgIoHtMgqHegpWAQ",
"UCw_LFe2pS8x3NyipGNJgeEA",
"UCLLw7jmFsvfIVaUFsLs8mlQ",
"UCDybamfye5An6p-j1t2YMsg",
"UC7cs8q-gJRlGwj4A8OmCmXg",
"UCQiNyL7ik4FIlV2UCvojq0g",
"UCJQJAI7IjbLcpsjWdSzYz0Q",
"UCu1xbgCV5o48h_BYCQD7KJg",
"UCggXQcy57bwxUeZI1LB8DCw",
"UCPcyffOFc7YGyDgqmvZJs7Q",
"UCC8nkE-p-7S0ZPJXrEs2QNA",
"UCcJQ96WlEhJ0Ve0SLmU310Q",
"UCXqKK0ZZO5EhS6USGkGcuiw",
"UCLpovxJVLBZrXJGCymB6LYw",
"UC2fYCeDDpXrQVjSPdJY4lyA",
"UC4MZ7zUHb5eAxU75Dc_nqdQ",
"UC8butISFwT-Wl7EV0hUK0BQ",
"UCsvqVGtbbyHaMoevxPAq9Fg",
"UCilJYqsETFZ7NG9g1xVKfcQ",
"UCsLo154Krjwhoz8W00N8ItA",
"UCsd0xmaXyUGpD35IX4xrYEw",
"UCv3N90AXQxUtaSnKSqOoOpA",
"UCmTS2gGo4gjZLt8LxVnF3XA",
"UCTHAIlSOqquwOKnd_nCJgAQ",
"UCWI-ohtRu8eEeDj93hmUsUQ",
"UC50JofjMZpr3KCbOx6dHi_A",
"UCUOMtzQmEmhIAF8G_cha_Xw",
"UCdngmbVKX1Tgre699-XLlUA",
"UC5qEPWtHLFRIjhW_3xd5g2A",
"UC2UXDak6o7rBm23k3Vv5dww",
"UCyUBW72KU30dfAYWLVNZO8Q",
"UCVe8CMJF4caRzuckVYV8CaQ",
"UCvDqylEzCZJHNea6k1XMkQw",
"UCEDkO7wshcDZ7UZo17rPkzQ",
"UCt-Pwe2fODjH4Wuwf5VqE7A",
"UClLXKYEEM8OBBx85DOa6-cg",
"UCN0u98NPeC0AW3pD6MbVeNg",
"UCwAnu01qlnVg1Ai2AbtTMaA",
"UCthRsjtSd2ZTrfgjgnBXMPw",
"UCx3ZYdyVr8BUQDIhS1QggxA",
"UCo2avEF03abQmCXdU_eUGkg",
"UC4xKdmAXFh4ACyhpiQ_3qBw",
"UCIrOHHdl6C4IWv96N0EaHMA",
"UCJNUtbsYnt-8meFSj_yF50Q",
"UCEHFikgnRuLd1HYKTLrae9Q",
"UC1nBZreWJArdFgiIu3BTeGw",
"UC3kFmMxz3IdLdz4TxA9a4DQ",
"UCpa0VHHeSUV1D17X8MoX5xA"]

In [7]:
len(channel_ids)

50

In [8]:
# Get credentials and create an API client
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

In [9]:
# channel request (getting all upload playlist ids from 50 selected tech career focused channels)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_ids)
)
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [10]:
# get upload playlists

upload_ids = [item["contentDetails"]["relatedPlaylists"]["uploads"] for item in response["items"]]

In [11]:
upload_ids_test = ["UU3kFmMxz3IdLdz4TxA9a4DQ", "UUilJYqsETFZ7NG9g1xVKfcQ", "UUN0u98NPeC0AW3pD6MbVeNg"]

In [12]:
video_ids = []
temp = []

In [13]:
for upload_playlist_id in upload_ids:
    
    #initial request
    request = youtube.playlistItems().list(
    part=['snippet'], 
    maxResults=50, 
    playlistId=upload_playlist_id)  #id=','.join(upload_ids))
    response = request.execute()

    #try to get all of the video id's from the response
    try:
        temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        video_ids = video_ids + temp
    except Exception as e:
        print(e)

    #increment page token
    next_page_token = response.get('nextPageToken')
    #print(next_page_token)
    #next_page_token
    
    # https://stackoverflow.com/questions/30653208/retrieve-video-ids-contained-in-a-playlist-youtube-api-v3
    # while there are still pages in the response to parse
    while next_page_token is not None: #and len(video_ids) < 100:
        #print(next_page_token)
        #make the same request, just get the next page of uploads (using nextPageToken which stores the token of the next page of search results
        request = youtube.playlistItems().list(
        part=['snippet'], 
        maxResults=50, 
        playlistId=upload_playlist_id,
        #id=','.join(upload_ids_test),
        pageToken = next_page_token)#id=','.join(upload_ids))

        response = request.execute()
        #make a temporary list of video ids on this page
        try:
            temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        except Exception as e:
            print(e)

        # add to video id list
        video_ids = video_ids + temp

        #increment page token
        next_page_token = response.get('nextPageToken')
        #print(next_page_token)

In [14]:
len(video_ids)

26034

In [25]:
def concatenate_df(response_json, output_df):    
    response_breakdown = {"snippet": ["channelTitle", "Published At", "channelId", "title", "categoryId", "description", "tags"], "contentDetails" : ["duration", "dimension", "definition", "caption", "licensedContent", "projection"], "statistics": ["viewCount", "likeCount", "favoriteCount", "commentCount"]}
    
    for video in response_json:
#"Tags": video["snippet"]["tags"],\
#                      #"Tag Count": len(video["snippet"]["tags"]), 
        
        row_to_add = {}
        
        #video id, if this isnt there we definitely need to just skip to the next, other parameters can be null
        try:
            row_to_add["id"] = video["id"]
        except KeyError:
            continue
        
        #for each category
        for category, category_list in response_breakdown.items():
            
            #for each feature
            for column_name in category_list:
                try:
                    row_to_add[column_name] = video[category][column_name]
                except KeyError:
                    row_to_add[column_name] = None
        
        row_to_add["Tag Count"] = len(row_to_add["tags"]) if row_to_add["tags"] != None else None
        row_to_add["Description Length (characters)"] = len(row_to_add["description"]) if row_to_add["description"] != None else None
        row_to_add["Description Length (words)"] = len(video["snippet"]["description"].split(" ")) if row_to_add["description"] != None else None
        
    
    
    
    
#         # for each feature in snippets
#         for column_name in response_breakdown["snippets"]:
#             try:
#                 row_to_add[column_name] = video["snippet"][column_name]
#             except KeyError:
#                 row_to_add[column_name] = None
        
#         # for each feature in contentDetails
#         for column_name in response_breakdown["content_details"]:
#             try:
#                 row_to_add[column_name] = video["contentDetails"][column_name]
#             except KeyError:
#                 row_to_add[column_name] = None
        
#         # for each feature in statistics
#         for column_name in response_breakdown["stats"]:
#             try:
#                 row_to_add[column_name] = video["statistics"][column_name]
#             except KeyError:
#                 row_to_add[column_name] = None
        
            
            
    # "Video ID":video["id"], "Channel Title": video["snippet"]["channelTitle"],\
    #                   "Published At": video["snippet"]["publishedAt"], "Channel ID": video["snippet"]["channelId"], \
    #                   "Title": video["snippet"]["title"],"Category ID": video["snippet"]["categoryId"], "Description Length (words)": len(video["snippet"]["description"].split(" ")), \
    #                   "Description Length (characters)": len(video["snippet"]["description"]), "Duration": video["contentDetails"]["duration"], "Dimension": video["contentDetails"]["dimension"], \
    #                  "Definition": video["contentDetails"]["definition"], "Caption": video["contentDetails"]["caption"], "Licensed Content (T/F)": video["contentDetails"]["licensedContent"], \
    #                  "Projection": video["contentDetails"]["projection"], "View Count": video["statistics"]["viewCount"], "Like Count": video["statistics"]["likeCount"], \
    #                  "Favorite Count": video["statistics"]["favoriteCount"], "Comment Count": video["statistics"]["commentCount"]}

        output_df = pd.concat( [ output_df, pd.DataFrame.from_records([row_to_add]) ], ignore_index=True )
    return output_df

In [16]:
final_df = pd.DataFrame()

In [24]:
#https://blog.enterprisedna.co/how-to-write-a-list-to-csv-in-python/
#save video id's to file just in case
with open('video_ids.csv', 'w', newline='') as file:
    # Step 4: Using csv.writer to write the list to the CSV file
    writer = csv.writer(file)
    writer.writerow(video_ids) # Use writerow for single list

In [18]:
video_ids[0]

'2LSPOgAtZeg'

In [267]:
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id='2ruoagytC7w'
    )
    
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [ ]:
rows = ["Video ID", "Video Title", "Channel Title", "Tags", "Tag Count", "Published At", "Channel ID", "Category ID", "Description Length (words)", "Description Length (characters)", "Duration", "Dimension", "Definition", "Caption", "Licensed Content (T/F)", "Projection", "View Count", "Like Count", "Favorite Count", "Comment Count"]

In [ ]:
rows_2 = ["id", "title", "channelTitle", "tags", "publishedAt", "channelId", "categoryId", "description", "duration", "dimension", "definition", "caption", "licensedContent", "projection", "viewCount", "likeCount", "favoriteCount", "commentCount"]

In [27]:
#video statistics request

# request = youtube.videos().list(
#     part="snippet,contentDetails,statistics",
#     id=','.join(video_ids[0:len(video_ids)-619])
# )


for i in range(0, len(video_ids), 50):
    
#     #to avoid list index out of bounds error (dataset not divisible by 50)
    #increment = i+50 if (i+50 < len(video_ids)-1) else len(video_ids)-1
    
    
    if i+50 < len(video_ids)-1:
        increment = i+50
    else:
        increment = len(video_ids)-1
    request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id=','.join(video_ids[i:increment])
    )
    
    response = request.execute()


    final_df = concatenate_df(response["items"], final_df)
    # print(i)
    
    if i % 1000 == 0:
        print(i)
    # print(increment)

final_df   

0


,id,channelTitle,Published At,channelId,title,categoryId,description,tags,duration,dimension,...,caption,licensedContent,projection,viewCount,likeCount,favoriteCount,commentCount,Tag Count,Description Length (characters),Description Length (words)
0,2LSPOgAtZeg,Dorian Develops,None,UCUOMtzQmEmhIAF8G_cha_Xw,Physical Health is Mental Health 💪,27,My Resume & Cover Letter Bundle (https://sowl....,None,PT31S,2d,...,false,True,rectangular,681,28,0,0,None,299,32
1,ftUc1Zxr_Q4,Dorian Develops,None,UCUOMtzQmEmhIAF8G_cha_Xw,I Turned Myself into a Business and It Made Me...,27,I productized myself and became a one-person b...,None,PT12M2S,2d,...,false,True,rectangular,2674,242,0,56,None,367,41
2,7hv2bb4vxHQ,Dorian Develops,None,UCUOMtzQmEmhIAF8G_cha_Xw,Morning Routines Are Out of Control!,27,I stumbled across the MOST RIDICULOUS morning ...,None,PT6M2S,2d,...,false,True,rectangular,3512,281,0,66,None,396,49
3,RoguKNx--v8,Dorian Develops,None,UCUOMtzQmEmhIAF8G_cha_Xw,Feeling LOST in your 20s? Advice I wish I had..,27,I had no direction in my 20s now I'm almost 40...,None,PT21M54S,2d,...,false,True,rectangular,3733,355,0,67,None,388,51
4,OLUSjr9oFTg,Dorian Develops,None,UCUOMtzQmEmhIAF8G_cha_Xw,100 Burpees and a Cold Plunge at 9pm on a Thur...,27,Day 4 of me doing a 100 burpees everyday for 3...,None,PT9M43S,2d,...,false,True,rectangular,1361,63,0,22,None,459,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,xKdtP3IZbnU,Alex The Analyst,None,UC7cs8q-gJRlGwj4A8OmCmXg,How I Manage My Time Working 60+ Hours Per Wee...,27,I have a fairly busy schedule and today I'm go...,"[Data Analyst, Data Analyst job, Data Analyst ...",PT43M30S,2d,...,false,True,rectangular,11594,324,0,161,10,1659,178
650,GsAXOr715AU,Alex The Analyst,None,UC7cs8q-gJRlGwj4A8OmCmXg,200k Livestream! | Shaving | Puppies | Giveaway,27,200k Livestream!\n\nI want to hear how this ch...,"[Data Analyst, Data Analyst job, Data Analyst ...",PT1H21M32S,2d,...,false,False,rectangular,3165,134,0,11,6,1970,212
651,2JnlQ8jwWbE,Alex The Analyst,None,UC7cs8q-gJRlGwj4A8OmCmXg,Things I HATE About Being a Data Analyst,27,Things I HATE About Being a Data Analyst.\n\nT...,"[Data Analyst, Data Analyst job, Data Analyst ...",PT10M34S,2d,...,false,True,rectangular,33196,1408,0,195,10,1790,203
652,hNTNZ00cZig,Alex The Analyst,None,UC7cs8q-gJRlGwj4A8OmCmXg,Best Website to Practice Technical Interview Q...,27,If you have a a data analyst interview coming ...,"[Data Analyst, Data Analyst job, Data Analyst ...",PT21M45S,2d,...,false,True,rectangular,17191,635,0,55,12,1779,189


In [28]:
final_df.to_csv('video_data.csv')  

In [248]:
def concatenate_df(response_json, output_df):
    for video in response_json:


        row_to_add = {"Video ID":video["id"], "Channel Title": video["snippet"]["channelTitle"],\ #"Tags": video["snippet"]["tags"],\
                      "Tag Count": len(video["snippet"]["tags"]), "Published At": video["snippet"]["publishedAt"], "Channel ID": video["snippet"]["channelId"], \
                      "Title": video["snippet"]["title"],"Category ID": video["snippet"]["categoryId"], "Description Length (words)": len(video["snippet"]["description"].split(" ")), \
                      "Description Length (characters)": len(video["snippet"]["description"]), "Duration": video["contentDetails"]["duration"], "Dimension": video["contentDetails"]["dimension"], \
                     "Definition": video["contentDetails"]["definition"], "Caption": video["contentDetails"]["caption"], "Licensed Content (T/F)": video["contentDetails"]["licensedContent"], \
                     "Projection": video["contentDetails"]["projection"], "View Count": video["statistics"]["viewCount"], "Like Count": video["statistics"]["likeCount"], \
                     "Favorite Count": video["statistics"]["favoriteCount"], "Comment Count": video["statistics"]["commentCount"]}

        output_df = pd.concat( [ output_df, pd.DataFrame.from_records([row_to_add]) ], ignore_index=True )
    return output_df

In [220]:
final_df

,Video ID,Channel Title,Tags,Tag Count,Published At,Channel ID,Title,Category ID,Description Length (words),Description Length (characters),Duration,Dimension,Definition,Caption,Licensed Content (T/F),Projection,View Count,Like Count,Favorite Count,Comment Count
0,OfAcLjgQUcQ,PushToProd,"[Software, Networking, NOC, network analyst, C...",36,2019-08-03T05:07:07Z,UCN0u98NPeC0AW3pD6MbVeNg,Day In the Life of an IT System Admin (Security),22,89,451,PT13M4S,2d,hd,false,True,rectangular,84849,1939,0,230


In [41]:
# # videos by keyword request
# temp = []
# # Get credentials and create an API client

# request = youtube.search().list(
#     part="snippet",
#     maxResults=50,
#     q="tech bootcamp"
# )
# response = request.execute()

# #get first 50 video id's and create list
# try:
#     video_ids = [video["id"]["videoId"] for video in response["items"]]
# except Exception as e:
#     print(e)

# #increment page token
# next_page_token = response.get('nextPageToken')

# # while there are still pages in the response to parse
# while next_page_token is not None and len(video_ids) < 100:
#     request = youtube.search().list(
#     part="snippet",
#     maxResults=51,
#     q="tech bootcamp",
#     pageToken = next_page_token # get results from the page after previous request
# )
    
#     #make a temporary list of video ids on this page
#     try:
#         temp = [video["id"]["videoId"] for video in response["items"]]
#     except Exception as e:
#         print(e)
    
#     # add to video id list
#     video_ids = video_ids + temp
    
#     #increment page token
#     next_page_token = response.get('nextPageToken')

# JSON(response)

# video_ids

In [81]:
# try:
#     video_ids = [video["id"]["videoId"] for video in response["items"]]
# except Exception as e:
#     print(e)

In [43]:
# #video statistics request

# request = youtube.videos().list(
#     part="snippet,contentDetails,statistics",
#     id=','.join(video_ids)
# )
# response = request.execute()

# JSON(response)

In [44]:
# likes = [video["statistics"]["likeCount"] for video in response["items"]]
# likes

In [45]:
# #channel id by name
# request = youtube.channels().list(
#     part="snippet,contentDetails,statistics",
#     forUsername="@JeffSu"
# )
# response = request.execute()

# JSON(response)

In [ ]:
# request = youtube.channels().list(
#     part="snippet,contentDetails,statistics",
#     id="UC_x5XG1OV2P6uZZ5FSM9Ttw"
# )
# response = request.execute()

# print(response)